In [70]:
# client.indices.delete(index='care_index')

ObjectApiResponse({'acknowledged': True})

In [1]:

pip install -qU elasticsearch sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [53]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from getpass import getpass

model = SentenceTransformer("all-MiniLM-L6-v2")

In [32]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
# cloud_id = 88c2be7fa9bf4c79a9df975bd451371f:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQwNWJmMGVmZmY0NGI0NGFjOWI3YTBkNTY3OTRhNDM3MCRkMjUwM2JkNzYxODg0NGM3ODZiNDk5NzFiY2IzMzE3YQ==
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
# api_key = YlNYZ2lJNEJaT0tRdWFXQ01BR186ODU1MVVUck5UeHFVbUVVOGh5Rk53dw==
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

In [71]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '05bf0efff44b44ac9b7a0d56794a4370', 'cluster_uuid': 'JEdCBHx9Rpu1t5gZAlRTTA', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [72]:
# Define the mapping
mappings = {
    "properties": {
        # "name_vector": {
        #     "type": "dense_vector",
        #     "dims": 384,
        #     "index": "true",
        #     "similarity": "cosine",
        # },
        # "short_description_vector": {
        #     "type": "dense_vector",
        #     "dims": 384,  # Assuming this is the dimensionality of the vector for the other field
        #     "index": True,
        #     "similarity": "cosine",
        # },
        "long_description_vector": {
            "type": "dense_vector",
            "dims": 384,  # Assuming you concatenate two vectors of dimension 384
            "index": True,
            "similarity": "cosine",
        },
        # "combined_vector": {
        #     "type": "dense_vector",
        #     "dims": 384,  # Assuming you concatenate two vectors of dimension 384
        #     "index": True,
        #     "similarity": "cosine",
        # }
    }
}

# Create the index
client.indices.create(index="care_index", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'care_index'})

In [73]:
# import json
# from urllib.request import urlopen

# url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/data.json"
# response = urlopen(url)
# books = json.loads(response.read())
# books

care_data = [
    {
    'name': 'Wellness / Sick Visit Codes',
    'short_description': 'Provides wellness and sick visit codes traditionally.',
    'long_description': 'Provides tables of mostly CPT codes related to wellness and sick visits. There are a couple of ICD-10-CM and even a modifier. Each code(s) has a description attached.',
    'type': 'Dashboard & Reports',
    'source': 'Github',
    'therapeutic_area': 'Oncology',
    'dataset':'Claims - Optum',
    'created_date': '2021-12-18'
    },
    {
    'name': 'Lynparza Ovarian Cancer Market Definition',
    'short_description': 'Codes to identify OC patients  and their attributes from claims data.',
    'long_description': 'Contains list of diagnosis codes for identifying OC and metastatic patients, CPT Codes for BRCA/HRD Testing.',
    'type': 'Tools',
    'source': 'Figma',
    'therapeutic_area': 'Vaccines',
    'dataset':'HCP Data',
    'created_date': '2022-01-29'
    },
    {
    'name': 'RCC RWD rulebook',
    'short_description': 'Secondary data in RCC',
    'long_description': 'Provides needed ICD, CPT, J, NCD code related to RCC in secondary data as well as rules on regimen hierarchy.',
    'type': 'Market Definitions',
    'source': 'Confluence',
    'therapeutic_area': 'Chronic Care',
    'dataset':'Claims - IQVIA',
    'created_date': '2022-07-02'
    },
    {
    'name': 'cSCC Model',
    'short_description': 'Python Package for cSCC Model',
    'long_description': 'Provides end users with a financial forecast and reports based on specified inputs.',
    'type': 'Tools',
    'source': 'Sharepoint',
    'therapeutic_area': 'Vaccines',
    'dataset':'Account Data',
    'created_date': '2022-10-25'
    },
    {
    'name': 'IQVIA RCC Cancer Patient Cohort',
    'short_description': 'Get cohort patient for RCC Cancer Primary Tumor.',
    'long_description': 'Get cohort patient for RCC Cancer using PTL logic from IQVIA datasource.',
    'type': 'ML Models',
    'source': 'Figma',
    'therapeutic_area': 'PAN TA',
    'dataset':'Source Agnostic',
    'created_date': '2022-12-22'
    }
    
]

# Iterate over each document and create the combined field
for document in care_data:
    name = document.get("name", "")
    short_description = document.get("short_description", "")
    long_description = document.get("long_description", "")
    
    # Combine the values of name, short_description, and long_description into a single string
    combined = f"{name} {short_description} {long_description}"
    
    # Add the combined field to the document
    document["combined"] = combined



In [74]:
operations = []
for drug in care_data:
    operations.append({"index": {"_index": "care_index"}})
   
    # name_vector = model.encode(drug["name"]).tolist()
    # short_description_vector = model.encode(drug["short_description"]).tolist()
    long_description_vector = model.encode(drug["long_description"]).tolist()
    # combined_vector = model.encode(drug["combined"]).tolist()
    
    # Combining the vectors
    # combined_vector = name_vector + short_description_vector + long_description_vector
    
    # Assigning the combined vector to the document
    # drug["name_vector"] = name_vector
    # drug["short_description_vector"] = short_description_vector
    drug["long_description_vector"] = long_description_vector
    # drug["combined_vector"] = combined_vector
    
    operations.append(drug)
client.bulk(index="care_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 115, 'items': [{'index': {'_index': 'care_index', '_id': 'oJS1iY4B8KzuOcexJFDh', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'care_index', '_id': 'oZS1iY4B8KzuOcexJFDh', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'care_index', '_id': 'opS1iY4B8KzuOcexJFDh', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'care_index', '_id': 'o5S1iY4B8KzuOcexJFDh', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_

In [75]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            print(hit['_score'], end=" - ")    
            print(hit['_source']['name'])

#####
index: Specifies the index or indices to search within. In this case, it's searching within the "care_index" index.

knn: This parameter indicates that the search operation will use the KNN search feature.

field: Specifies the field in the documents of the index where the KNN search will be performed. Here, it's set to "long_description_vector".

query_vector: Specifies the vector to search for nearest neighbors. It seems that this vector is generated by encoding the text "rcc cancer" using some model (perhaps a word embedding model or some other vectorization technique).

k: Specifies the number of nearest neighbors to retrieve. In this case, it's set to 1, meaning it will retrieve the single nearest neighbor.

num_candidates: Specifies the number of candidate nearest neighbors to consider during the search. This parameter can help improve search performance by limiting the number of documents considered during the search.

In [81]:
# Taking input from the user
user_input = input("Please enter something: ")

response = client.search(
    index="care_index",
    query= {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": user_input,
                        "fields": ["name^3", "short_description^2", "long_description"],
                        "fuzziness": "AUTO"
                    }
                }
            ],
            "should": [
                {"regexp": {"name": f".*{user_input}.*"}},
                {"regexp": {"short_description": f".*{user_input}.*"}},
                {"regexp": {"long_description": f".*{user_input}.*"}}
            ],
            "must": [{"match": {'type':'ml models'}}],
            "must": [{"match": {'source':'figma'}}],
            "must": [{"match": {'therapeutic_area':'pan ta'}}],
            "must": [{"match": {'dataset':'source agnostic'}}],
            "must": [{"range": {"created_date": {"gte": "2022-10-30"}}}],
        
        }
    },
    # knn={
    #     "field": "long_description_vector",
    #     "query_vector": model.encode(user_input),
    #     "k": 10,
    #     "num_candidates": 100,
    # },
)


pretty_response(response)

4.0 - IQVIA RCC Cancer Patient Cohort
